## Ethereum Blockchain Transaction Analysis

### Link to GitHub Repository
https://github.com/cs418-fa24/project-check-in-team-2

## Project Introduction
In this project, we are analyzing Ethereum blockchain transaction data to investigate trends and insights related to recent transactions.
Our primary focus is to explore transaction patterns, fees, and the frequency of various transaction types to gain a deeper understanding of blockchain dynamics.

This analysis involves collecting recent transactions, cleaning the data, performing exploratory data analysis (EDA), and applying machine learning techniques to gain insights.

### Key Questions:
1. What patterns or trends can be identified in Ethereum transactions?
2. Are there specific metrics that can help predict Ethereum price movements?

## Changes to Project Scope
Since the initial proposal, we have made the following changes:
- **Scope Addition**: We have included a deeper analysis of transaction fees and patterns by block height to identify temporal patterns.
- **Scope Reduction**: We initially planned to analyze more historical data; however, we have limited the scope to recent blocks due to time constraints.

These adjustments will allow us to focus on recent trends and complete the analysis within the project timeline.